### To run this model yourself: go to Run -> Run All Cells in the top left menu bar.

In [ ]:
from epx import Job, ModelConfig, SynthPop

import plotly.express as px
import plotly.graph_objects as go

import plot_mitigations
import time

# Exploring the Utility of Mitigation Strategies for Respiratory Disease

### In this notebook, we will model the spread of an influenza through Kewaunee County, WI.

Our influenza model uses an SEIR structure. If you'd like to learn more about disease transmission in FRED, check out Lesson 7 in the Quickstart Guide.

For now, here's a brief overview of the model:
- We begin with 10 agents randomly exposed to the disease. 
- Exposed agents become Infectious after an average of two days, and they can either be Symptomatic (66%) or Asymptomatic (33%). 
- Agents remain infectious for an average of 5 days, during which they can transmit the disease to other susceptible agents that they come into contact with. 
After the infectious period, they recover and are no longer susceptible to reinfection.

In [ ]:
flu_config = ModelConfig(
    synth_pop=SynthPop("US_2010.v5", ["Kewaunee_County_WI"]),
    start_date="2023-01-01",
    end_date="2023-07-01",
)

flu_job = Job(
    "model/main.fred",
    config=[flu_config],
    key="cl_flu_job",
    fred_version="11.0.1",
    results_dir="/home/epx/cl-results"
)

flu_job.execute()

# the following loop idles while we wait for the simulation job to finish and periodically prints an update
update_count = 0
update_interval = 3
start_time = time.time()
timeout   = 300 # timeout in seconds
idle_time = 20   # time to wait (in seconds) before checking status again
while str(flu_job.status) != 'DONE':
    if str(flu_job.status) == 'ERROR':
        logs = flu_job.status.logs
        log_msg = "; ".join(logs.loc[logs.level == "ERROR"].message.tolist())
        print(f"Job failed with the following error:\n '{log_msg}'")
        break
    if time.time() > start_time + timeout:
        msg = f"Job did not finish within {timeout / 60} minutes."
        raise RuntimeError(msg)
    
    if update_count >= update_interval:
        update_count = 0
        print(f"Job is still processing after {time.time() - start_time:.0f} seconds")
        
    update_count += 1
    
    time.sleep(idle_time)

print(f"Job completed in {time.time() - start_time:.0f} seconds")

str(flu_job.status)

## Exploring the Model Output

In [ ]:
baseline_states = plot_mitigations.get_states(flu_job)
baseline_exposures = plot_mitigations.get_explocs(flu_job)
baseline_locations = plot_mitigations.get_expmap_data(flu_job)

In [ ]:
fig = go.Figure()

for x in ["Exposed", "InfectiousA", "InfectiousS", "Recovered"]:
    fig.add_trace(
        go.Scatter(
            x=baseline_states["sim_date"],
            y=baseline_states[x],
            mode="lines",
            line=go.scatter.Line(width=3),
            showlegend=True,
            name=x,
        )
    )

fig.update_layout(
    font_family="Epistemix Label",
    yaxis_title="New infections per day",
    xaxis_title="Date",
    legend_title="State",
    xaxis_range=["2023-01-01", "2023-04-01"],
    hovermode="x",
    height=450,
)

fig.show()

Our influenza model tracks how many agents are newly exposed, infectious, or recovered each day. In the figure above, we can see that it takes about two weeks from the initial seeding of exposures at the beginning of the simulation for community spread to increase substantially, and that the disease has run its course after about 10 weeks.

Below we plot an animation of infections colored by the type of location at which the exposure occurred (Household, Workplace, etc)  for the duration of the simulation.

In [ ]:
plot_mitigations.plot_animation_by_exposure_location(baseline_exposures)

Here we show the same data as in the above animation, but summarized into an interactive map with the size of the circles representing households, workplaces, and schools scaled proportionally to the number of agent-to-agent transmission events that took place in those locations throughout the simulation. This helps us visually locate places that experienced unusually high numbers of infections.

In [ ]:
fig = px.scatter_mapbox(
    baseline_locations,
    lat="my_exp_lat",
    lon="my_exp_lon",
    color="ExposureLocation",
    color_discrete_sequence=["#F0438D", "#2BD6AF", "#A76FF4"],
    opacity=0.8,
    size="exp_scale",
    size_max=12,
    zoom=9.25,
    height=600,
    hover_data={"exp_scale": False, "NumberExposed": True},
)

fig.update_layout(
    mapbox_style=plot_mitigations.MAPSTYLE, mapbox_accesstoken=plot_mitigations.TOKEN
)
fig.update_layout(margin={"r": 0, "t": 60, "l": 0, "b": 0})
fig.update_layout(
    title="Sites of Influenza Exposure",
    title_font_size=24,
)
fig.update_layout(legend_title_text="Exposure location type")
fig.show()

Here we show another animated map of the same simulation data, this time color-coded by the demographic attributes of the agents that were infected.

In [ ]:
plot_mitigations.plot_animation_by_demog_group(baseline_exposures)

The corresponding time series of infections for the above animation is shown below.

In [ ]:
plot_mitigations.plot_time_series_by_demog_group(baseline_exposures)

In [ ]:
# deleting our job now that we are done with it
flu_job.delete(interactive=False)

FRED is a poweful tool not just for understanding disease spread in the aggregate but also for exploring the individual behaviors that are contributing factors. As part of our baseline model, agents recorded where they were exposed to the respiratory disease.